# Exchange Trade Protocol

In investory must consider factors such as downside risk, market conditions, and the length of time it will take for each investment to realize returns. They also need to consider opportunity costs that represent the potential benefits that an individual, investor, or business misses out on when choosing the ETP over another.

The _Return On Investment_ (ROI) of a single investment is the net price gain from holding the asset by the asset's original cost. The cost of an asset includes not only the purchase price, but also any commissions, management fees, or other expenses associated with the acquisition. __ETP ROI__ is the differential gain (or loss) of the sum of the weighted original market capital to that of the current sum of the weighted market capital.

* Let ${X}$ = {${x_0}$, ${x_1}$, ..., ${x_n-1}$} the set of crypto currency assets 
* respectively the set of asset market capitalization at time ${t}$: ${M_t(X)}$ = {${M_t(x_i)}$ | ${0 \le i \le n-1}$}
* the top ${N}$ market capitalization assets at time ${t}$: ${M_t(\overline{X})}$ = {${M_t({x}_j)}$ | ${j \in \left[0,N-1\right] \subset \left[0,n-1\right]}$}
* The fund allocation weights for the top ${N}$ assets at time ${t}$: ${W_t(\overline{X})}$ = {${w_{t}(x_{j})}$ | ${j \in \left[0,N-1\right] \subset \left[0,n-1\right]}$}; such that $\sum_{j=0}^{N-1} {w_{t}(x_{j}) = 1.0}$
* Weighted market fund allocation at time ${t}$: ${F_{t}(\overline{X})}$  = $\sum_{j = 0}^{n-1} {w_{t}(x_{j})} \times {M_{t}(x_{j})}$
* __Objective__ is ${Max(F_{t}(\overline{X})) > R_{t}}$ by finding the optimal set of weights ${W_t(\overline{X})}$; where ${R_{t}}$ is the ROI from a risk free investment of the fund

## Modern Portfolio Theorem
The [post-modern portfolio theory](https://www.investopedia.com/terms/p/pmpt.asp) (PMPT) attempts to improve on modern portfolio theory by minimizing downside risk instead of variance.

In [5]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('default')     #switch betweeb 'default' and 'ignore'

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

# Fund allocation and back testing

* [Estimate portfolio weights using Monte Carlo simulation](https://medium.com/analytics-vidhya/how-to-estimate-optimal-stock-portfolio-weights-using-monte-carlo-simulations-modern-portfolio-d27d534e8a1a)
* [Portfolio selection using Bayes Theorem](https://www.hindawi.com/journals/mpe/2019/4246903/)
* Introduction to [Modern Portfolio Theory](https://www.quantconnect.com/tutorials/introduction-to-financial-python/modern-portfolio-theory)
* [Practical Portfolio Optimisation](https://pythoninvest.com/long-read/practical-portfolio-optimisation)


1. Define the start and end date of the simulation cycle
   1. start_date: assumed date fund is allocated
   1. end_date: pseudo current date
1. Get the market capitalization data for entire time period
   1. Compute retrospective mean and standard deviation upto start_date
      1. compute for entire retrospective period
      1. compute for past 7, 20, and 100 days
   1. Estimate next day return based on simple returns formula
      1. estimate based on 7, 30, and 100 day intervals
      1. compare with the actual value of the following day
      1. estimate ratios based on USD & BTC as risk free returns
1. Get top N assets for each day to compute and continue from start-date to end-date
   1. fund allocation weights based on market capital expected returns
   1. Re-balance the weights after estimating the returns 

## Load marketcap data

In [142]:
import sys
sys.path.insert(1, '../lib')

if debug:
    import importlib
    etl = importlib.reload(etl)

''' Initialize the dataETL class '''
path = "../data/market_cap_2021-01-01_2022-06-01/"
cls_etl = etl.ExtractLoadTransform(dataPath=path)

In [134]:
import dataETL as etl
import pandas as pd

''' Get list of data file names '''
_l_fnames = cls_etl.get_file_list()
print("Retrieved %d files in dir: %s " % (len(_l_fnames),path))
''' Load data into dataframe '''
rec_marketcap_df = cls_etl.load_data(_l_fnames)
print("Loaded %d rows %s" % (rec_marketcap_df.shape[0],str(rec_marketcap_df.columns)))
''' Transform data with coin ids in columns '''
piv_marketcap_df = cls_etl.transfrom_data(rec_marketcap_df)
print("Transformed dataframe %d rows %s" % (piv_marketcap_df.shape[0],str(piv_marketcap_df.columns)))
print("Data load and transformation to pivot table complete!")


Retrieved 63 files in dir: ../data/market_cap_2021-01-01_2022-06-01/ 
Loaded 3637 rows Index(['Date', 'ID', 'Symbol', 'market_cap'], dtype='object')
Transformed dataframe 524 rows Index(['Date', 'bitcoin', 'cardano', 'ethereum', 'litecoin', 'ripple',
       'solana'],
      dtype='object')
Data load and transformation to pivot table complete!


## Filtered by date range

In [135]:
import datetime

start_dt = datetime.datetime(2022,1,1)
end_dt = datetime.datetime(2022,2,1)

''' >= start-date '''
if isinstance(start_dt, datetime.date):
    mask = (rec_marketcap_df["Date"] >= start_dt)
    rec_marketcap_df = rec_marketcap_df[mask]
    mask = (piv_marketcap_df["Date"] >= start_dt)
    piv_marketcap_df = piv_marketcap_df[mask]
''' <= end-date '''
if isinstance(end_dt, datetime.date):
    mask = (rec_marketcap_df["Date"] <= end_dt)
    rec_marketcap_df = rec_marketcap_df[mask]
    mask = (piv_marketcap_df["Date"] <= end_dt)
    piv_marketcap_df = piv_marketcap_df[mask]
print(rec_marketcap_df.info())
print(piv_marketcap_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 224 entries, 0 to 31
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        224 non-null    datetime64[ns]
 1   ID          224 non-null    object        
 2   Symbol      224 non-null    object        
 3   market_cap  224 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 8.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 365 to 396
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      32 non-null     datetime64[ns]
 1   bitcoin   32 non-null     float64       
 2   cardano   32 non-null     float64       
 3   ethereum  32 non-null     float64       
 4   litecoin  32 non-null     float64       
 5   ripple    32 non-null     float64       
 6   solana    32 non-null     float64       
dtypes: dateti

## Instantiate ETP class

In [136]:
import sys
sys.path.insert(1, '../lib')
import assetETP as etp

if debug:
    import importlib
    etp = importlib.reload(etp)

data_name = "coindecks"
cls_etp = etp.ExchangeTradeProtocol(name=data_name)
print(dir(cls_etp))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__main__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'days_offset', 'get_simple_returns', 'get_topN_assets', 'name', 'p_val', 'pd', 'rolling_mean', 'rolling_stdv', 'sum_weighted_returns', 'weights_matrix', 'window_length']


## Top-N Assets

In [ ]:
import numpy as np
topN = 3
_weights = np.array(cls_etp.weights_matrix(N=topN,S=10)).reshape(-1,topN)
_topNassets_df = cls_etp.get_topN_assets(rec_marketcap_df, N=topN)
_topNassets_df=_topNassets_df.reindex()

## Actual returns

In [137]:
actual_returns = cls_etp.sum_weighted_returns(_topNassets_df, _weights)

## Rolling mean and standard deviation

In [138]:
rolling_marketcap_mean = cls_etp.rolling_mean(piv_marketcap_df, period=7)
transp_rolling_marketcap_mean = cls_etl.transpose_pivot(rolling_marketcap_mean)
rolling_marketcap_stdv = cls_etp.rolling_stdv(piv_marketcap_df, period=7)
transp_rolling_marketcap_stdv = cls_etl.transpose_pivot(rolling_marketcap_stdv)

## Reshape dataframes to match TopN

In [150]:
import sys
sys.path.insert(1, '../lib')

if debug:
    import importlib
    etl = importlib.reload(etl)

''' Initialize the dataETL class '''
path = "../data/market_cap_2021-01-01_2022-06-01/"
cls_etl = etl.ExtractLoadTransform(dataPath=path)

In [151]:
matching_rolling_market_cap_mean = cls_etl.match_dataframes(_topNassets_df,transp_rolling_marketcap_mean)

[Error]Class <ExchangeTradeProtocol> Function <match_dataframes> Cannot perform 'rand_' with a dtyped [object] array and scalar of type [bool]
Traceback (most recent call last):
  File "/home/gnewy/.local/lib/python3.8/site-packages/pandas/core/ops/array_ops.py", line 301, in na_logical_op
    result = op(x, y)
  File "/home/gnewy/.local/lib/python3.8/site-packages/pandas/core/roperator.py", line 52, in rand_
    return operator.and_(right, left)
TypeError: unsupported operand type(s) for &: 'int' and 'str'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/gnewy/.local/lib/python3.8/site-packages/pandas/core/ops/array_ops.py", line 315, in na_logical_op
    result = libops.scalar_binop(x, y, op)
  File "pandas/_libs/ops.pyx", line 210, in pandas._libs.ops.scalar_binop
  File "/home/gnewy/.local/lib/python3.8/site-packages/pandas/core/roperator.py", line 52, in rand_
    return operator.and_(right, left)
TypeError: uns

In [164]:
from datetime import date, timedelta

pred_marketcap_df = piv_marketcap_df.copy()
num_days = (end_dt - start_dt).days + 1
sharp = (_07_day_roll_simp_return_mean[_07_day_roll_simp_return_mean['Date'] == end_dt][_l_coin_ids] - 0.02/365)/ \
    _07_day_roll_simp_return_stdv[_07_day_roll_simp_return_stdv['Date'] == end_dt][_l_coin_ids]
#for now_date in (start_dt + timedelta(n) for n in range(num_days)):
#    print(now_date)
print(sharp)
print(simple_returns[simple_returns['Date'] == end_dt][_l_coin_ids])

      bitcoin   cardano  ethereum  litecoin   ripple    solana
334 -0.023394 -0.385817  0.184135 -0.077693 -0.19041 -0.145532
      bitcoin   cardano  ethereum  litecoin    ripple    solana
334 -0.014434 -0.033933  0.042531  0.003433  0.005133  0.019014


## Simple returns

In [138]:
simple_returns_df = cls_etp.get_simple_returns(market_df)
#_l_cols = [col for col in simple_returns_df if col !='Date']
#simple_returns_df[_l_cols] = simple_returns_df[simple_returns_df[_l_cols] < 0][_l_cols]
_title = "Expected Returns from "+str(_min_date)+" to "+str(_max_date)
fig = px.line(simple_returns_df, x="Date", y=simple_returns_df.columns,
              hover_data={"Date": "|%B %d, %Y"},
              title=_title)
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.show()

/home/gnewy/.local/lib/python3.8/site-packages/plotly/io/_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



## DEPRECATED